# Obtain activity
The main objective of this task is to label each day consumption depending on its activity. We want to model two types of days based on the consumption activity of each building:
- **Active** day. Usual consumption during working days.
- **Inactive** day. Usual consuomption during holidays.

This will be achieved by performing k-means (for 2 clusters), every building by its own.

After that, different types of consumers will be tagged, based on percentiles: high consumers, medium consumers and low consumers in the case of active consumptions; high consumers and low consumers, in the case of inactive consumptions.

In [1]:
CONS_PATH = 'C:/Users/thmas/OneDrive - Universidad de Castilla-La Mancha/Informática/TFG/out/'

In [2]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [3]:
raw = pd.read_csv(CONS_PATH + 'raw_consumptions.csv', index_col='day', converters={'consumptions': lambda x: list(map(float, x.strip('[]').split()))}, na_values='nan', parse_dates=True, infer_datetime_format=True)
raw

,building_id,weekday,consumptions
day,,,
2011-07-26,27,1,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
2011-07-27,27,2,"[17.0, 19.0, 18.35079465, 35.84631282, 47.8462..."
2011-07-28,27,3,"[18.88870418, 18.80300889, 18.84589226, 35.845..."
2011-07-29,27,4,"[20.0, 21.0, 20.0, 37.78877899, 45.84570469, 5..."
2011-07-30,27,5,"[17.29811321, 17.0, 17.23969745, 17.84583302, ..."
...,...,...,...
2020-05-10,2233,6,"[8.69106633, 8.54560612, 8.13895298, 8.8050705..."
2020-05-11,2233,0,"[8.57166388, 10.30621946, 10.4596744, 12.97239..."
2020-05-12,2233,1,"[8.31569322, 9.21411531, 10.19029213, 12.96810..."


In [4]:
counter_id = 487 # Counter ID example

raw_df = raw[raw['building_id'] == counter_id]
raw_df

,building_id,weekday,consumptions
day,,,
2013-12-17,487,1,"[nan, nan, nan, nan, nan, nan, nan, 4.49733527..."
2013-12-18,487,2,"[12.10293213, 12.10293213, 12.10293213, 12.102..."
2013-12-19,487,3,"[11.26490906, 11.26490906, 11.26490906, 11.264..."
2013-12-20,487,4,"[10.9838824, 10.9838824, 10.9838824, 10.983882..."
2013-12-21,487,5,"[6.93115242, 7.59915394, 7.59915394, 7.5991539..."
...,...,...,...
2020-05-23,487,5,"[10.0, 10.0, 9.0, 10.0, 10.0, 10.0, 10.4483297..."
2020-05-24,487,6,"[10.42261925, 9.0, 9.07036963, 10.92963037, 9...."
2020-05-25,487,0,"[11.0, 9.0, 9.01754998, 12.98245002, 12.0, 12...."


In [5]:
raw_df['total_cons'] = raw_df['consumptions'].apply(np.nansum)
raw_df['total_cons']

D:\PythonEnvironments\pytfg\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


day
2013-12-17    198.144249
2013-12-18    278.172099
2013-12-19    262.029504
2013-12-20    225.807653
2013-12-21    181.711693
                 ...    
2020-05-23    240.577381
2020-05-24    242.422619
2020-05-25    245.000000
2020-05-26    249.406628
2020-05-27    245.593372
Name: total_cons, Length: 2354, dtype: float64

Remove outliers

In [6]:
q3 = np.percentile(raw_df['total_cons'], 75)
q1 = np.percentile(raw_df['total_cons'], 25)

iqr = q3 - q1

maximum = np.percentile(raw_df['total_cons'], 75) + 1.5 * iqr
minimum = np.percentile(raw_df['total_cons'], 25) - 1.5 * iqr

raw_df = raw_df[(raw_df['total_cons'] > minimum) & (raw_df['total_cons'] < maximum)]

In [7]:
X = raw_df['total_cons'].values.reshape(-1, 1)

scaler = StandardScaler()

X = scaler.fit_transform(X)

km = KMeans(n_clusters=2).fit(X)

In [8]:
raw_df.insert(2, 'active', True)
for i in range(raw_df.shape[0]):
    if km.cluster_centers_[0] < km.cluster_centers_[1]:
        if km.labels_[i] == 0:
            raw_df['active'].iloc[i] = False
    else:
        if km.labels_[i] == 1:
            raw_df['active'].iloc[i] = False
            
raw_df

D:\PythonEnvironments\pytfg\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
D:\PythonEnvironments\pytfg\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,building_id,weekday,active,consumptions,total_cons
day,,,,,
2013-12-17,487,1,False,"[nan, nan, nan, nan, nan, nan, nan, 4.49733527...",198.144249
2013-12-18,487,2,True,"[12.10293213, 12.10293213, 12.10293213, 12.102...",278.172099
2013-12-19,487,3,True,"[11.26490906, 11.26490906, 11.26490906, 11.264...",262.029504
2013-12-20,487,4,False,"[10.9838824, 10.9838824, 10.9838824, 10.983882...",225.807653
2013-12-21,487,5,False,"[6.93115242, 7.59915394, 7.59915394, 7.5991539...",181.711693
...,...,...,...,...,...
2020-05-23,487,5,False,"[10.0, 10.0, 9.0, 10.0, 10.0, 10.0, 10.4483297...",240.577381
2020-05-24,487,6,False,"[10.42261925, 9.0, 9.07036963, 10.92963037, 9....",242.422619
2020-05-25,487,0,False,"[11.0, 9.0, 9.01754998, 12.98245002, 12.0, 12....",245.000000


Defining consumption type. 0 means lowest consumer

In [9]:
raw = pd.read_pickle(CONS_PATH + 'clean_consumptions.zip')
raw

,building_id,weekday,active,consumptions,total_cons
day,,,,,
2011-07-26,27,1,False,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",368.000000
2011-07-27,27,2,True,"[17.0, 19.0, 18.35079465, 35.84631282, 47.8462...",1376.111296
2011-07-28,27,3,True,"[18.88870418, 18.80300889, 18.84589226, 35.845...",1432.888704
2011-07-29,27,4,True,"[20.0, 21.0, 20.0, 37.78877899, 45.84570469, 5...",1374.701887
2011-07-30,27,5,False,"[17.29811321, 17.0, 17.23969745, 17.84583302, ...",451.298662
...,...,...,...,...,...
2020-05-10,2233,6,False,"[8.69106633, 8.54560612, 8.13895298, 8.8050705...",201.552366
2020-05-11,2233,0,False,"[8.57166388, 10.30621946, 10.4596744, 12.97239...",266.428660
2020-05-12,2233,1,False,"[8.31569322, 9.21411531, 10.19029213, 12.96810...",253.456566


In [10]:
buildings_df = pd.DataFrame()

for counter_id in raw['building_id'].unique():
    building = raw[raw['building_id'] == counter_id]
    
    actives = building['total_cons'][building['active']]
    inactives = building['total_cons'][building['active'] == False]
    
    buildings_df = buildings_df.append(pd.DataFrame({'building_id': counter_id, 'active': [True, False], 'mean_cons': [actives.mean(), inactives.mean()]}), ignore_index=True)
    
buildings_df

,building_id,active,mean_cons
0,27,True,1164.853429
1,27,False,522.576203
2,28,True,1133.045919
3,28,False,534.186961
4,37,True,1902.723098
...,...,...,...
189,2209,False,223.679921
190,2210,True,2002.971497
191,2210,False,284.642782
192,2233,True,481.260526


In [11]:
def get_consumption_type(df: pd.DataFrame, n: int):
    increment = 100 / n

    types = []
    for i in range(n):
        if i == n - 1:
            atype = df[df['mean_cons'] >= np.percentile(df['mean_cons'], increment * i)]
        else:
            atype = df[(df['mean_cons'] >= np.percentile(df['mean_cons'], increment * i)) & (df['mean_cons'] < np.percentile(df['mean_cons'], increment * (i + 1)))]

        atype['type'] = i

        types.append(atype)
        
    return pd.concat(types)

In [12]:
actives = get_consumption_type(buildings_df[buildings_df['active']], 3)
inactives = get_consumption_type(buildings_df[buildings_df['active'] == False], 2)

D:\PythonEnvironments\pytfg\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [13]:
types = pd.concat([actives, inactives])
types

,building_id,active,mean_cons,type
24,86,True,156.795554,0
26,88,True,134.334077,0
44,426,True,51.643651,0
48,436,True,64.594253,0
50,437,True,17.152971,0
...,...,...,...,...
185,2207,False,2156.218538,1
187,2208,False,344.124138,1
189,2209,False,223.679921,1
191,2210,False,284.642782,1


In [14]:
raw = raw.merge(types, on=['building_id', 'active'], how='left').set_index(raw.index)
raw = raw[['building_id', 'weekday', 'active', 'type', 'consumptions']]

raw

,building_id,weekday,active,type,consumptions
day,,,,,
2011-07-26,27,1,False,1,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
2011-07-27,27,2,True,2,"[17.0, 19.0, 18.35079465, 35.84631282, 47.8462..."
2011-07-28,27,3,True,2,"[18.88870418, 18.80300889, 18.84589226, 35.845..."
2011-07-29,27,4,True,2,"[20.0, 21.0, 20.0, 37.78877899, 45.84570469, 5..."
2011-07-30,27,5,False,1,"[17.29811321, 17.0, 17.23969745, 17.84583302, ..."
...,...,...,...,...,...
2020-05-10,2233,6,False,1,"[8.69106633, 8.54560612, 8.13895298, 8.8050705..."
2020-05-11,2233,0,False,1,"[8.57166388, 10.30621946, 10.4596744, 12.97239..."
2020-05-12,2233,1,False,1,"[8.31569322, 9.21411531, 10.19029213, 12.96810..."
